In [ ]:
# pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import xlsxwriter
import os
import sys

# ==========================================
# 0. CONFIGURACIÓN DE RUTAS
# ==========================================
PATH_LIGIE = r"C:/Users/Edward/Desktop/Bancomext/Tariffs/data/intermediate/LIGIE_Maestra_Unificada.xlsx"
PATH_HTS_INPUT = r"C:/Users/Edward/Desktop/Bancomext/Tariffs/data/raw/htsdata.csv"
PATH_OUTPUT = r"C:/Users/Edward/Desktop/Bancomext/Tariffs/data/intermediate/HTS_Procesada_Final.xlsx"

print("--- INICIANDO PROCESAMIENTO HTS (CON SECCIONES) ---")
print(f"Referencia LIGIE: {PATH_LIGIE}")
print(f"Datos HTS Input:  {PATH_HTS_INPUT}")
print(f"Archivo Salida:   {PATH_OUTPUT}")
print("-" * 50)

# ==========================================
# FUNCIONES AUXILIARES
# ==========================================

def forzar_dos_digitos(serie):
    return serie.fillna('').astype(str).str.replace(r'\.0$', '', regex=True).str.strip().str.zfill(2)

def forzar_un_digito(serie):
    return serie.fillna('').astype(str).str.replace(r'\.0$', '', regex=True).str.strip().str.zfill(1)

def formatear_hts_con_puntos(code_clean):
    if len(code_clean) != 8:
        return code_clean 
    return f"{code_clean[:4]}.{code_clean[4:6]}.{code_clean[6:8]}"

def limpiar_y_normalizar_hts(code):
    c = str(code).replace('.', '').strip()
    if len(c) % 2 != 0: c = '0' + c
    if len(c) >= 10: c = c[:8]
    return c

# ==========================================
# FASE 1: CARGAR DICCIONARIOS LIGIE (INCLUYENDO SECCIÓN)
# ==========================================

def cargar_diccionarios_ligie(path):
    if not os.path.exists(path):
        sys.exit(f"ERROR: No existe {path}")

    print(">> Cargando referencias LIGIE y mapeando Secciones...")
    try:
        df = pd.read_excel(path, sheet_name="Panel Extendido", dtype=str)
        
        # Recuperar componentes clave
        df['Ref_Cap'] = forzar_dos_digitos(df['Capítulo'])      
        df['Ref_Part_Suffix'] = forzar_dos_digitos(df['Partida']) 
        df['Ref_Desdob'] = forzar_un_digito(df['Desdob.'])      
        df['Ref_Sub'] = forzar_un_digito(df['Subpartida'])      

        # Construir Llaves
        df['Key_Cap'] = df['Ref_Cap']
        df['Key_Part'] = df['Ref_Cap'] + df['Ref_Part_Suffix']
        df['Key_Sub'] = df['Key_Part'] + "." + df['Ref_Desdob'] + df['Ref_Sub']
        
        # --- NUEVO: DICCIONARIOS DE SECCIÓN ---
        # Como cada Capítulo pertenece a una única Sección, creamos un df auxiliar único por capítulo
        df_caps = df[['Ref_Cap', 'Sección', 'Nombre Sección']].drop_duplicates(subset=['Ref_Cap'])
        
        ref_sec_rom = pd.Series(df_caps['Sección'].values, index=df_caps['Ref_Cap']).to_dict()
        ref_sec_nom = pd.Series(df_caps['Nombre Sección'].values, index=df_caps['Ref_Cap']).to_dict()

        # --- DICCIONARIOS ESTÁNDAR ---
        ref_cap = pd.Series(df['Nombre Capítulo'].values, index=df['Key_Cap']).to_dict()
        ref_part = pd.Series(df['Nombre Partida'].values, index=df['Key_Part']).to_dict()
        ref_subpart = pd.Series(df['Nombre Subpartida'].values, index=df['Key_Sub']).to_dict()
        ref_desdob_name = pd.Series(df['Nombre Desdoblamiento'].values, index=df['Key_Sub']).to_dict()

        print("   Diccionarios listos.")
        return ref_sec_rom, ref_sec_nom, ref_cap, ref_part, ref_subpart, ref_desdob_name

    except Exception as e:
        sys.exit(f"ERROR leyendo LIGIE: {e}")

REF_SEC_ROM, REF_SEC_NOM, REF_CAP, REF_PART, REF_SUB, REF_DESDOB = cargar_diccionarios_ligie(PATH_LIGIE)

# ==========================================
# FASE 2: LIMPIEZA INICIAL HTS
# ==========================================

def limpiar_hts_inicial(path):
    if not os.path.exists(path):
        sys.exit(f"ERROR: No existe {path}")
    
    print(">> Procesando HTS...")
    try:
        if path.endswith('.csv'):
            df = pd.read_csv(path, dtype=str)
        else:
            df = pd.read_excel(path, dtype=str)
    except:
        sys.exit("No se pudo leer el archivo.")

    cols = ["HTS Number", "Description", "Unit of Quantity", "General Rate of Duty", "Special Rate of Duty"]
    df = df[cols].copy().fillna('')
    
    # Columna temporal
    df['HTS_Clean_Temp'] = df['HTS Number'].str.replace('.', '', regex=False).str.strip()
    
    # Corte 99
    mask_99 = df['HTS_Clean_Temp'].str.startswith('99', na=False)
    if mask_99.any():
        idx = df[mask_99].index[0]
        df = df.iloc[:idx].copy()
    
    # Eliminar temporal
    df = df.drop(columns=['HTS_Clean_Temp'])
    
    return df

DF_HTS_CLEAN = limpiar_hts_inicial(PATH_HTS_INPUT)

# ==========================================
# FASE 3: PREPARACIÓN Y NORMALIZACIÓN
# ==========================================
print(">> Normalizando códigos HTS...")

df_panels = DF_HTS_CLEAN[DF_HTS_CLEAN['General Rate of Duty'] != ''].copy()
df_panels['HTS_8_Clean'] = df_panels['HTS Number'].apply(limpiar_y_normalizar_hts)
df_panels['HTS_Dotted'] = df_panels['HTS_8_Clean'].apply(formatear_hts_con_puntos)

# ==========================================
# FASE 4: CONSTRUCCIÓN DE PANELES
# ==========================================

clean = df_panels['HTS_8_Clean'] 
dotted = df_panels['HTS_Dotted'] 

# Claves para mapeo
key_cap = clean.str[:2]
key_part = clean.str[:4]
key_sub = clean.str[:4] + "." + clean.str[4:6]

# --- A. PANEL NUMÉRICO ---
df_num = pd.DataFrame()
df_num['Código']   = dotted
# Agregamos Sección (Romana) consultando por capítulo
df_num['Sección']  = key_cap.map(REF_SEC_ROM).fillna('') 
df_num['Capítulo'] = clean.str[:2]
df_num['Partida']  = clean.str[2:4]
df_num['Desdoblamiento'] = clean.str[4:5]
df_num['Subpartida']     = clean.str[5:6]
df_num['Fracción']       = clean.str[6:8]
df_num['Unit of Quantity'] = df_panels['Unit of Quantity']
df_num['General Rate of Duty'] = df_panels['General Rate of Duty']

# --- B. PANEL TEXTUAL ---
df_txt = pd.DataFrame()
df_txt['Código'] = dotted
# Agregamos Nombre Sección
df_txt['Nombre Sección'] = key_cap.map(REF_SEC_NOM).fillna('')
df_txt['Nombre Capítulo'] = key_cap.map(REF_CAP).fillna('')
df_txt['Nombre Partida']  = key_part.map(REF_PART).fillna('')
df_txt['Nombre Desdoblamiento'] = key_sub.map(REF_DESDOB).fillna('')
df_txt['Nombre Subpartida']     = key_sub.map(REF_SUB).fillna('')
df_txt['Descripción Fracción'] = df_panels['Description']
df_txt['Unit of Quantity'] = df_panels['Unit of Quantity']
df_txt['General Rate of Duty'] = df_panels['General Rate of Duty']

# --- C. PANEL EXTENDIDO ---
df_ext = pd.DataFrame()
df_ext['Código'] = dotted

# Sección
df_ext['Sección'] = df_num['Sección']
df_ext['Nombre Sección'] = df_txt['Nombre Sección']

# Capítulo
df_ext['Capítulo']        = df_num['Capítulo']
df_ext['Nombre Capítulo'] = df_txt['Nombre Capítulo']

# Partida
df_ext['Partida']        = df_num['Partida']
df_ext['Nombre Partida'] = df_txt['Nombre Partida']

# Desdoblamiento
df_ext['Desdoblamiento'] = df_num['Desdoblamiento']
df_ext['Nombre Desdoblamiento'] = df_txt['Nombre Desdoblamiento']

# Subpartida
df_ext['Subpartida']     = df_num['Subpartida']
df_ext['Nombre Subpartida'] = df_txt['Nombre Subpartida']

# Fracción
df_ext['Fracción']             = df_num['Fracción']
df_ext['Descripción Fracción'] = df_txt['Descripción Fracción']

df_ext['Unit of Quantity']     = df_panels['Unit of Quantity']
df_ext['General Rate of Duty'] = df_panels['General Rate of Duty']

# --- D. PANEL CONCATENADO ---
df_concat = pd.DataFrame()
df_concat['Código'] = dotted

def concatenar(row):
    partes = [
        row['Nombre Sección'],  # Ahora incluimos la Sección al inicio
        row['Nombre Capítulo'],
        row['Nombre Partida'],
        row['Nombre Desdoblamiento'],
        row['Nombre Subpartida'],
        row['Descripción Fracción']
    ]
    validas = [str(x).strip() for x in partes if str(x).strip() not in ['', 'ND', 'nan']]
    return " ".join(validas)

df_concat['Descripción Completa'] = df_txt.apply(concatenar, axis=1)
df_concat['Unit of Quantity']     = df_panels['Unit of Quantity']
df_concat['General Rate of Duty'] = df_panels['General Rate of Duty']

# ==========================================
# FASE 5: EXPORTACIÓN CON FORMATO AVANZADO
# ==========================================
print(f">> Guardando Excel: {PATH_OUTPUT}")

try:
    writer = pd.ExcelWriter(PATH_OUTPUT, engine='xlsxwriter')
    workbook = writer.book

    # Estilos
    fmt_header = workbook.add_format({
        'bold': True, 'border': 1, 'bg_color': '#BDD7EE',
        'valign': 'vcenter', 'align': 'center', 'font_name': 'Arial', 'font_size': 10, 'text_wrap': True
    })
    fmt_body = workbook.add_format({'border': 1, 'valign': 'top', 'font_name': 'Arial', 'font_size': 9})
    fmt_wrap = workbook.add_format({'border': 1, 'valign': 'top', 'font_name': 'Arial', 'font_size': 9, 'text_wrap': True})

    def escribir_hoja(df, nombre):
        df.to_excel(writer, sheet_name=nombre, index=False, startrow=1, header=False)
        ws = writer.sheets[nombre]
        
        # Headers
        for idx, col in enumerate(df.columns):
            ws.write(0, idx, col, fmt_header)
            
        # Determinar Ancho de Columnas de Texto según la Hoja
        if "Textual" in nombre or "Extendido" in nombre:
            ancho_texto = 37 
        else:
            ancho_texto = 50 

        # Cuerpo
        for col_idx, col_name in enumerate(df.columns):
            # Detectar columnas texto largo + Special Rate
            columnas_wrap = ["Description", "Descripción", "Nombre", "Completa", "Special Rate"]
            es_largo = any(x in col_name for x in columnas_wrap)
            
            estilo = fmt_wrap if es_largo else fmt_body
            
            # Escritura celda por celda
            for row_idx in range(len(df)):
                val = df.iloc[row_idx, col_idx]
                if pd.isna(val): val = ""
                ws.write(row_idx + 1, col_idx, val, estilo)
            
            # Ajustar Anchos
            if es_largo:
                ws.set_column(col_idx, col_idx, ancho_texto)
            elif "General Rate" in col_name:
                ws.set_column(col_idx, col_idx, 15)
            elif len(col_name) <= 10 and "Unit" not in col_name: 
                ws.set_column(col_idx, col_idx, 10)
            else:
                ws.set_column(col_idx, col_idx, 12)

    escribir_hoja(DF_HTS_CLEAN, "HTS Cleaned")
    escribir_hoja(df_num, "Panel Numérico")
    escribir_hoja(df_txt, "Panel Textual")
    escribir_hoja(df_ext, "Panel Extendido")
    escribir_hoja(df_concat, "Concatenado")

    writer.close()
    print("¡PROCESO FINALIZADO EXITOSAMENTE!")

except Exception as e:
    print(f"ERROR: {e}")

--- INICIANDO PROCESAMIENTO HTS (CON SECCIONES) ---
Referencia LIGIE: C:/Users/Edward/Desktop/Bancomext/Tariffs/data/intermediate/LIGIE_Maestra_Unificada.xlsx
Datos HTS Input:  C:/Users/Edward/Desktop/Bancomext/Tariffs/data/raw/htsdata.csv
Archivo Salida:   C:/Users/Edward/Desktop/Bancomext/Tariffs/data/intermediate/HTS_Procesada_Final.xlsx
--------------------------------------------------
>> Cargando referencias LIGIE y mapeando Secciones...
   Diccionarios listos.
>> Procesando HTS...
>> Normalizando códigos HTS...
>> Guardando Excel: C:/Users/Edward/Desktop/Bancomext/Tariffs/data/intermediate/HTS_Procesada_Final.xlsx
¡PROCESO FINALIZADO EXITOSAMENTE!


Debemos hacer un archivo markdown con un formato y estructura similar al de la LIGIE, únicamente tomando este script como referencia, pero tomando en cuenta las mismas prácticas consideradas para las funciones y sus nombres, así como la explicación individual de cada función. La idea es tomar la manera en que se documentó el archivo de la LIGIE y aplicar esa lógica a este.